In [1]:
import os
import tarfile
import ast
import json
import pandas as pd
import math
from collections import Counter
import numpy as np
from scipy.stats import entropy
import logging
import sys
import xml.etree.ElementTree as ET

logstd = logging.StreamHandler(sys.stdout)

logging.basicConfig(
    format="%(asctime)s %(levelname)s %(name)s:%(lineno)d - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S %Z",
    level=logging.INFO,
    handlers=[logstd]
)

log = logging.getLogger()

Extract files

In [ ]:
repo_dir = "/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples"
#Find all tar.gz files in the repository
tar_files = []
for root, dirs, files in os.walk(repo_dir):
    for file in files:
        if file.endswith(".tar.gz") or file.endswith(".tar.bz2") or file.endswith(".tar.xz"):
            tar_files.append(os.path.join(root, file))

#Unzip the files with different compression methods
for tar_file in tar_files:
    if tar_file.endswith(".tar.gz"):
        mode = 'r:gz'
    elif tar_file.endswith(".tar.bz2"):
        mode = 'r:bz2'
    elif tar_file.endswith(".tar.xz"):
        mode = 'r:xz'
    else:
        mode = 'r'
    
    try:
        with tarfile.open(tar_file, mode) as tar_ref:
            extract_dir = os.path.splitext(os.path.splitext(tar_file)[0])[0]
            tar_ref.extractall(extract_dir)
            print(f"Extracted {tar_file} to {extract_dir}")
    except (tarfile.ReadError, tarfile.CompressionError) as e:
        print(f"Failed to extract {tar_file}: {e}")

print("Extraction process completed.")

Count number of packages

In [ ]:
dataset_1_dir = "/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples"
dataset_2_dir = "/mnt/volume_nyc1_01/pypi_malregistry"  

def count_package_files(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".tar.gz") or file.endswith(".tar.bz2") or file.endswith(".tar.xz"):
                count += 1
    return count

count_1 = count_package_files(dataset_1_dir)
count_2 = count_package_files(dataset_2_dir)

print(f"Number of packages in dataset 1: {count_1}")
print(f"Number of packages in dataset 2: {count_2}")

Convert setup.py to JSON

In [ ]:
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)

def parse_setup_py(setup_py_path):
    setup_args = {}
    try:
        with open(setup_py_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Remove BOM if present
        if content.startswith('\ufeff'):
            content = content[1:]

        # Parse the content with AST
        tree = ast.parse(content, filename=setup_py_path)

        for node in tree.body:
            if (isinstance(node, ast.Expr) and isinstance(node.value, ast.Call) and
                    isinstance(node.value.func, ast.Name) and node.value.func.id == 'setup'):
                for kwarg in node.value.keywords:
                    try:
                        value = ast.literal_eval(kwarg.value)
                    except (ValueError, SyntaxError):
                        # Fallback to using the repr of the value if literal_eval fails
                        value = ast.dump(kwarg.value)
                    setup_args[kwarg.arg] = value

    except SyntaxError as e:
        log.error(f"SyntaxError in {setup_py_path}: {e}")
        return None

    return setup_args

def convert_setup_to_json(dataset_dir):
    setup_py_files = []
    for root, _, files in os.walk(dataset_dir):
        for file in files:
            if file == 'setup.py':
                setup_py_files.append(os.path.join(root, file))

    for setup_py in setup_py_files:
        setup_args = parse_setup_py(setup_py)
        if setup_args is None:
            continue  # Skip this setup.py file due to SyntaxError

        json_path = os.path.join(os.path.dirname(setup_py), 'setup.json')
        with open(json_path, 'w', encoding='utf-8') as json_file:
            json.dump(setup_args, json_file, indent=2)
        log.debug(f'Converted {setup_py} to {json_path}')

# Replace 'dataset_dir' with the variable containing your packages directory path
dataset_dir = '/mnt/volume_nyc1_01/benignPyPI'
convert_setup_to_json(dataset_dir)


Count files successfully converted to JSON

In [ ]:
dataset_1_dir = "/mnt/volume_nyc1_01/Backstabbers-Knife-Collection/samples"
dataset_2_dir = "/mnt/volume_nyc1_01/benignPyPI"

def is_valid_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            json.load(file)
        return True
    except (ValueError, json.JSONDecodeError):
        return False

def count_valid_json_files(directory):
    count = 0
    for root, _, files in os.walk(directory):
        for file in files:
            if file == "setup.json" and is_valid_json_file(os.path.join(root, file)):
                count += 1
    return count

count_1 = count_valid_json_files(dataset_1_dir)
count_2 = count_valid_json_files(dataset_2_dir)

print(f"Number of valid setup.json files in dataset 1: {count_1}")
print(f"Number of valid setup.json files in dataset 2: {count_2}")